# loading the text

In [23]:
from google.colab import drive

drive.mount('/content/drive')

# learning
learning_file_path = "wonderland.txt"
saving_weights_path = '/content/drive/MyDrive/semes_2/semester_2/advenced_mechine_learning/ex3/weights/'

# generating
generating_file_path = "wonderland.txt"
weights_file_path = "/content/drive/MyDrive/semes_2/semester_2/advenced_mechine_learning/ex3/weights/weights-improvement-50-1.0240-bigger.hdf5"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
text = 'Temperatures were measured with TAT-2000 and TAT-5000 TemporalScanner thermometers (Exergen Corp, Watertown, MA) and the Athena Elevated Temperature Detection System (Athena Security, Austin, TX). Exergen reports their instruments to be accurate within 0.2°C and 0.1°C, respectively.3,4 The Athena telethermographic system uses artificial intelligence to detect human faces by measuring the temperature of multiple points on the face relative to a blackbody temperature reference source.5 According to Athena Security, the system is accurate within 0.3°C.5 Systems were purchased from Athena Security. Accepting manufacturer specifications, detecting 0.2°C difference between devices (assuming standard deviation of ±0.3°C) required 26 measurements from each device. One subject was measured 104 times with 4 different TAT-2000s (26 measurements per device) and 104 times with 4 different TAT-5000s (26 measurements per device) by a single operator, and 13 times with the Athena system at a single location within 90 minutes to minimize subject and environmental temperature variation. We repeated measurements with the same subject and thermometer operator at a second location with 3 additional TAT-5000s, 1 TAT-5000 used previously (104 measurements, 26 per device) and a second thermal camera (13 measurements). We simulated fever using air-activated hand warmers (HotHands, Kobayashi Americas, Dalton, GA) held to the forehead. Descriptive statistical analyses were performed with Stata version 15 SE software (StataCorp, College Station, TX). Summaries were reported as means with 95% confidence intervals and differences were tested by 1-way ANOVA. A 2-sided P < .05 was considered statistically significant. During the first session, the TAT-2000s measured higher temperatures [mean, 98.3°F (95% CI, 98.2–98.3) or 36.8°C (95% CI, 36.8–36.8)] than the TAT-5000s [mean, 97.8 °F (95% CI, 97.8–97.9) or 36.6°C (95% CI, 36.5–36.6)] or the Athena system [mean, 97.9°F (95% CI, 97.8–98.0) or 36.6°C (95% CI, 36.5–36.7)] (P < .05). There was no significant difference between the TAT-5000s and the Athena system [mean difference, −0.07°F (95% CI, −0.23 to 0.09) or −0.04°C (95% CI, −0.13 to 0.05)], but the TAT-2000s measured temperatures significantly higher than the Athena [mean difference, 0.40°F (95% CI, 0.24–0.56) or 0.22°C (95% CI, 0.13–0.31)]. During the second testing session, the TAT-5000s measured 0.34°F (95% CI, 0.20–0.48) or 0.19°C (95% CI, 0.11–0.26) [mean, 98.1°F (95% CI, 98.1–98.2) or 36.7°C (95% CI, 36.7–36.8)] higher than the Athena system [mean, 97.8°F (95% CI, 97.7–97.9) or 36.6°C (95% CI, 36.5–36.6)] (P < .05). HotHands warmers reached up to 46.1°C (115°F) 15–30 minutes after activation and were held at the forehead. A “symptomatic” individual in single-file line 6 feet (2 m) between “normal” individuals passing the camera at a rate of 1 individual per second, was detected in 8 of 8 attempts. Additionally, when the forehead was warmed and the warmer then removed, the Athena system was able to detect temperatures of >99°F or 37.2°C (Fig. 1) in 5 of 5 attempts. \n\n Screeners using TAT-5000s took a median of 16.5 seconds from the start of taking the temperature through cleaning the device until the thermometer was ready again. The Athena system has no effective delay from person to person passing in a single-file line. The COVID-19 pandemic has led to the implementation of temperature screening in a wide variety of facilities. Although temperature screening has been used in public settings during previous infectious diseases outbreaks,6–8 the usefulness of temperature screening to detect potential infections has been questioned.6,7 However, temperature screening may discourage symptomatic individuals from entering public places and may increase comfort for healthy people.8\n Our study using noninvasive devices was not designed to test the accuracy of devices, though temporal scanners are widely considered reliable enough for professional use.9,10 In our use, temperatures measured by telethermographic systems were similar to those obtained by temporal scanners, suggesting similar performance. Cost is the biggest barrier to implementation for telethermographic systems. For our investment recovery analysis, we considered turnaround time difference between temporal scanners and a thermal camera for each screened individual at a high-entry location with large groups arriving in a short period, desired throughput rate of 1 person per second, maintaining 6-feet (2-m) social distancing and single-file lines for individual symptom screening. We estimated needing 6 temporal scanner operators for every 1 thermal camera operator. With our organization’s direct labor rates and overhead costs, investment recovery was estimated to occur in months, leading to adoption of 4 telethermographic systems at our 2 highest-entry locations. We reduced screening staff from 24 to 4 individuals, and there are now no waiting lines at these locations. In conclusion, our experience demonstrates that a telethermographic system improves screening throughput and reports temperatures similar to those recorded by temporal scanners, with acceptable investment recovery time.'

# Learning larger LSTM Network to Generate Text for Alice in Wonderland


In [11]:
	
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
raw_text = open(learning_file_path, 'r', encoding='utf-8').read()
#raw_text = text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath=f"{saving_weights_path}/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Total Characters:  5210
Total Vocab:  55
Total Patterns:  5110
Epoch 1/50
80/80 [==============================] - 99s 1s/step - loss: 3.4079

Epoch 00001: loss improved from inf to 3.29542, saving model to /content/drive/MyDrive/semes_2/semester_2/advenced_mechine_learning/ex3/weights/weights-improvement-01-3.2954-bigger.hdf5
Epoch 2/50
80/80 [==============================] - 96s 1s/step - loss: 3.2416

Epoch 00002: loss improved from 3.29542 to 3.23941, saving model to /content/drive/MyDrive/semes_2/semester_2/advenced_mechine_learning/ex3/weights/weights-improvement-02-3.2394-bigger.hdf5
Epoch 3/50
80/80 [==============================] - 95s 1s/step - loss: 3.2171

Epoch 00003: loss improved from 3.23941 to 3.20493, saving model to /content/drive/MyDrive/semes_2/semester_2/advenced_mechine_learning/ex3/weights/weights-improvement-03-3.2049-bigger.hdf5
Epoch 4/50
80/80 [==============================] - 94s 1s/step - loss: 3.1631

Epoch 00004: loss improved from 3.20493 to 3.16245,

# Load Larger LSTM network and generate text

In [38]:

import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "wonderland.txt"
#raw_text = open(filename, 'r', encoding='utf-8').read()
txt = 'Methods for coordinating data-gathering with standardized nomenclature, Methods for coordinating data-gathering with standardized nomenclature'
txt = text = ' and the Athena Elevated Temperature Detection System (Athena Security, Austin, TX). Exergen reports their instruments to be accurate within 0.2°C and 0.1°C, respectively.3,4 The Athena telethermographic system uses artificial intelligence to detect human faces by measuring the temperature of multiple points on the face relative to a blackbody temperature reference source.5 According to Athena Security, the system is accurate within 0.3°C.5 Systems were purchased from Athena Security. Accepting manufacturer specifications, detecting 0.2°C difference between devices (assuming standard deviation of ±0.3°C) required 26 measurements from each device. One subject was measured 104 times with 4 different TAT-2000s (26 measurements per device) and 104 times with 4 different TAT-5000s (26 measurements per device) by a single operator, and 13 times with the Athena system at a single location within 90 minutes to minimize subject and environmental temperature variation. We repeated measurements with the same subject and thermometer operator at a second location with 3 additional TAT-5000s, 1 TAT-5000 used previously (104 measurements, 26 per device) and a second thermal camera (13 measurements). We simulated fever using air-activated hand warmers (HotHands, Kobayashi Americas, Dalton, GA) held to the forehead. Descriptive statistical analyses were performed with Stata version 15 SE software (StataCorp, College Station, TX). Summaries were reported as means with 95% confidence intervals and differences were tested by 1-way ANOVA. A 2-sided P < .05 was considered statistically significant. During the first session, the TAT-2000s measured higher temperatures [mean, 98.3°F (95% CI, 98.2–98.3) or 36.8°C (95% CI, 36.8–36.8)] than the TAT-5000s [mean, 97.8 °F (95% CI, 97.8–97.9) or 36.6°C (95% CI, 36.5–36.6)] or the Athena system [mean, 97.9°F (95% CI, 97.8–98.0) or 36.6°C (95% CI, 36.5–36.7)] (P < .05). There was no significant difference between the TAT-5000s and the Athena system [mean difference, −0.07°F (95% CI, −0.23 to 0.09) or −0.04°C (95% CI, −0.13 to 0.05)], but the TAT-2000s measured temperatures significantly higher than the Athena [mean difference, 0.40°F (95% CI, 0.24–0.56) or 0.22°C (95% CI, 0.13–0.31)]. During the second testing session, the TAT-5000s measured 0.34°F (95% CI, 0.20–0.48) or 0.19°C (95% CI, 0.11–0.26) [mean, 98.1°F (95% CI, 98.1–98.2) or 36.7°C (95% CI, 36.7–36.8)] higher than the Athena system [mean, 97.8°F (95% CI, 97.7–97.9) or 36.6°C (95% CI, 36.5–36.6)] (P < .05). HotHands warmers reached up to 46.1°C (115°F) 15–30 minutes after activation and were held at the forehead. A “symptomatic” individual in single-file line 6 feet (2 m) between “normal” individuals passing the camera at a rate of 1 individual per second, was detected in 8 of 8 attempts. Additionally, when the forehead was warmed and the warmer then removed, the Athena system was able to detect temperatures of >99°F or 37.2°C (Fig. 1) in 5 of 5 attempts. \n\n Screeners using TAT-5000s took a median of 16.5 seconds from the start of taking the temperature through cleaning the device until the thermometer was ready again. The Athena system has no effective delay from person to person passing in a single-file line. The COVID-19 pandemic has led to the implementation of temperature screening in a wide variety of facilities. Although temperature screening has been used in public settings during previous infectious diseases outbreaks,6–8 the usefulness of temperature screening to detect potential infections has been questioned.6,7 However, temperature screening may discourage symptomatic individuals from entering public places and may increase comfort for healthy people.8\n Our study using noninvasive devices was not designed to test the accuracy of devices, though temporal scanners are widely considered reliable enough for professional use.9,10 In our use, temperatures measured by telethermographic systems were similar to those obtained by temporal scanners, suggesting similar performance. Cost is the biggest barrier to implementation for telethermographic systems. For our investment recovery analysis, we considered turnaround time difference between temporal scanners and a thermal camera for each screened individual at a high-entry location with large groups arriving in a short period, desired throughput rate of 1 person per second, maintaining 6-feet (2-m) social distancing and single-file lines for individual symptom screening. We estimated needing 6 temporal scanner operators for every 1 thermal camera operator. With our organization’s direct labor rates and overhead costs, investment recovery was estimated to occur in months, leading to adoption of 4 telethermographic systems at our 2 highest-entry locations. We reduced screening staff from 24 to 4 individuals, and there are now no waiting lines at these locations. In conclusion, our experience demonstrates that a telethermographic system improves screening throughput and reports temperatures similar to those recorded by temporal scanners, with acceptable investment recovery time.'
raw_text = txt.lower()
print(raw_text)
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
print(dataY)
y = np_utils.to_categorical(dataY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# load the network weights
model.load_weights(weights_file_path)

model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

 and the athena elevated temperature detection system (athena security, austin, tx). exergen reports their instruments to be accurate within 0.2°c and 0.1°c, respectively.3,4 the athena telethermographic system uses artificial intelligence to detect human faces by measuring the temperature of multiple points on the face relative to a blackbody temperature reference source.5 according to athena security, the system is accurate within 0.3°c.5 systems were purchased from athena security. accepting manufacturer specifications, detecting 0.2°c difference between devices (assuming standard deviation of ±0.3°c) required 26 measurements from each device. one subject was measured 104 times with 4 different tat-2000s (26 measurements per device) and 104 times with 4 different tat-5000s (26 measurements per device) by a single operator, and 13 times with the athena system at a single location within 90 minutes to minimize subject and environmental temperature variation. we repeated measurements w

In [ ]:
# to add -


ascii_values = ascii_values[np.where(ascii_values < 128)]